<a href="https://colab.research.google.com/github/isaramadunika/Advanced_Statistics_Project/blob/main/Advanced_Statistics_for_Data_Science.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')

# Read the CSV file
df = pd.read_csv('/content/drive/MyDrive/ASDS/Dialog_2009_2023_Daily.csv')

# Display the first few rows of the dataset
print("First 5 rows of the dataset:")
print(df.head())

# Check the shape of the dataset
print("\
Dataset shape (rows, columns):", df.shape)

# Check data types and missing values
print("\
Data types and non-null counts:")
print(df.info())

# Check for missing values
print("\
Missing values per column:")
print(df.isnull().sum())

First 5 rows of the dataset:
        Date  Price  Open  High   Low     Vol. Change %
0   7/8/2009   5.25  5.25  5.25  5.00  714.20K    0.00%
1   7/9/2009   5.25  5.25  5.25  5.00  386.70K    0.00%
2  7/13/2009   5.25  5.25  5.25  5.00  290.90K    0.00%
3  7/14/2009   5.25  5.25  5.25  5.25  163.60K    0.00%
4  7/15/2009   5.25  5.25  5.25  5.00   63.10K    0.00%
Dataset shape (rows, columns): (3268, 7)
Data types and non-null counts:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3268 entries, 0 to 3267
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Date      3268 non-null   object 
 1   Price     3268 non-null   float64
 2   Open      3268 non-null   float64
 3   High      3268 non-null   float64
 4   Low       3268 non-null   float64
 5   Vol.      3268 non-null   object 
 6   Change %  3268 non-null   object 
dtypes: float64(4), object(3)
memory usage: 178.8+ KB
None
Missing values per column:
Date        0
Price